# Manteniéndose informado sobre accidentes en Ciudad de México

Primero buscamos los cinco periodicos más populares en la Ciudad de México. Según [Prensa escrita](https://www.prensaescrita.com/america/df.php) son los siguientes:

- Reforma
- La Prensa
- El Universal
- El Gráfico
- Capital México.

Podemos utilizar BeatifulSoup para buscar cada link en los sitios de cada periódico y crear un archivo CSV con la noticia y el día en que se publicó.
Haciendo un poco de limpieza, podemos crear un vocabulario con palabras que ocurran en noticias de accidentes, e iterar en cada noticia para buscar coincidencias y almacenar las noticias.

Por último se puede programar el script para ejecutarse semanalmente usando la herramienta CRON en Linux o Task Scheduler en Windows.

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import date
import re
import pandas as pd
import string

In [2]:
domainlist = ["https://www.reforma.com/", "https://www.la-prensa.com.mx/", "https://www.eluniversal.com.mx/", "https://www.elgrafico.mx/", "https://www.capitalmexico.com.mx/"]

In [3]:
for d in domainlist:
    news = requests.get(d)
    newspaper = re.findall(r'://www.([\w\-\.]+)',d)[0]
    content = news.content
    parser = BeautifulSoup(content, 'html.parser')
    
    with open('news-dataset.csv', "w", encoding="utf-8") as csv_file:
        columns = ['date', 'newspaper', 'newstitle']
        writer = csv.DictWriter(csv_file, fieldnames=columns)
        c = 1
        row = {}
        writer.writeheader()
        for news in parser.find_all('a'):
            row['date'] = date.today()
            row['newspaper'] = newspaper
            row['newstitle'] = news.text
            writer.writerow(row)

In [4]:
news_data = pd.read_csv('news-dataset.csv', encoding= 'unicode_escape')

In [5]:
news_data.head()

,date,newspaper,newstitle
0,2020-11-09,capitalmexico.com.mx,\r\nCapital MÃ©xico\r\n
1,2020-11-09,capitalmexico.com.mx,\r\n\r\n
2,2020-11-09,capitalmexico.com.mx,Inicio
3,2020-11-09,capitalmexico.com.mx,Nacional
4,2020-11-09,capitalmexico.com.mx,Estados


In [6]:
news_data = news_data.dropna()

In [7]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
def normalize(string):
    string = string.lower()
    string = re.sub("[^A-Za-z0-9\s]", "", string)
    string = re.sub("\s+", " ", string)
    
    return string

In [9]:
news_data['normalized'] = news_data['newstitle'].apply(normalize)

In [10]:
news_data.head()

,date,newspaper,newstitle,normalized
0,2020-11-09,capitalmexico.com.mx,\r\nCapital MÃ©xico\r\n,capital mxico
1,2020-11-09,capitalmexico.com.mx,\r\n\r\n,
2,2020-11-09,capitalmexico.com.mx,Inicio,inicio
3,2020-11-09,capitalmexico.com.mx,Nacional,nacional
4,2020-11-09,capitalmexico.com.mx,Estados,estados


In [11]:
news_data = news_data.reset_index(drop=True)

In [12]:
news_data.head()

,date,newspaper,newstitle,normalized
0,2020-11-09,capitalmexico.com.mx,\r\nCapital MÃ©xico\r\n,capital mxico
1,2020-11-09,capitalmexico.com.mx,\r\n\r\n,
2,2020-11-09,capitalmexico.com.mx,Inicio,inicio
3,2020-11-09,capitalmexico.com.mx,Nacional,nacional
4,2020-11-09,capitalmexico.com.mx,Estados,estados


In [13]:
mask = (news_data['newstitle'].str.len() >= 20)
news_data = news_data.loc[mask]

In [14]:
news_data.head()

,date,newspaper,newstitle,normalized
23,2020-11-09,capitalmexico.com.mx,PolicÃ­as de CancÃºn detonan armas durante man...,policas de cancn detonan armas durante manifes...
24,2020-11-09,capitalmexico.com.mx,Coahuila se declara en sesiÃ³n permanente para...,coahuila se declara en sesin permanente para e...
25,2020-11-09,capitalmexico.com.mx,ConfÃ­an diputados en evitar que se elimine el...,confan diputados en evitar que se elimine el f...
26,2020-11-09,capitalmexico.com.mx,La elecciÃ³n en EEUU tal vez matice algunos as...,la eleccin en eeuu tal vez matice algunos aspe...
27,2020-11-09,capitalmexico.com.mx,"âHija de pâ¦â le dice diputada de Morena,...",hija de p le dice diputada de morena irma juan...


In [15]:
news_data['date'] = pd.to_datetime(news_data['date'])

In [16]:
vocabulary = ['muere', 'muri', 'murieron', 'accidente', 'accident', 'accidentaron', 'choca', 'choc', 'chocaron', 'herido', 'herido', 'heridas']

In [17]:
news_list = []

for index, row in news_data.iterrows():
    
    new = row['normalized']
    split_new = row['normalized'].split()
  
    for word in split_new:
        if word in vocabulary:
            news_list.append(news_data.loc[index, ['date', 'newspaper', 'newstitle']])
        

In [18]:
news_list

[date                                       2020-11-09 00:00:00
 newspaper                                 capitalmexico.com.mx
 newstitle    A balazos muriÃ³ el peridosita Israel VÃ¡zquez...
 Name: 41, dtype: object,
 date                                       2020-11-09 00:00:00
 newspaper                                 capitalmexico.com.mx
 newstitle    Accidente de Mexibus en la MÃ©xico-Pachuca dej...
 Name: 61, dtype: object,
 date                                  2020-11-09 00:00:00
 newspaper                            capitalmexico.com.mx
 newstitle    MuriÃ³ el escritor Sandro Cohen por COVID-19
 Name: 88, dtype: object,
 date                                       2020-11-09 00:00:00
 newspaper                                 capitalmexico.com.mx
 newstitle    MuriÃ³ Vera Lynn, la Â«noviaÂ» de los soldados...
 Name: 118, dtype: object]